# CSCI316 Individual Assignment 1
# Thawdar Swe Zin (8039276)
## (Task 2)
---


In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import random
import math
from pprint import pprint 

In [2]:
# Creating dataframes
churnTest_df = pd.read_csv("customer_churn_dataset-testing-master.csv")
churnTrain_df = pd.read_csv("customer_churn_dataset-training-master.csv")

In [6]:
len(churnTest_df)

64374

In [7]:
len(churnTrain_df)

440833

### 1) Implement three DT models by the split criteria of Information Gain, Gain Ratio and Gini Index. You can use either binary-split or multiple-split

#### We will be using binary splits, ie: == or != statements, we will need a function to determine the optimal partioning

The following functions are implemented to achieve our objectives: 

    1. Function for calculating information gain, gini index,and gain ratio
    2. Function for checking if we have reached a leaf node (stopping criteria)
    3. Function for grabbing the class one we have reached a leaf node
    4. Function for getting all possible splits
    5. (3) functions for determining best split (info gain, gain index, gain ratio)



##### Function for calculating information gain, gini index,and gain ratio  

In [8]:
# calculate entropy for the dataset (for testing/ verification)
label_col = churnTrain_df.iloc[:,-1]
_, n_counts = np.unique(label_col, return_counts=True)

prob = n_counts / n_counts.sum()
entropyOfDataset = sum(prob * -np.log2(prob))

print("Entropy of dataset: ", entropyOfDataset)

Entropy of dataset:  0.9870102753005665


# Function for binary splitting

In [17]:
def split_data(data, split_col, split_val):
    split_col_val = data[:, split_col]

    left = data[split_col_val == split_val]
    right = data[split_col_val != split_val]

    return left, right

#### Function for calculating best split based on information gain

In [90]:
def calculate_entropy(data):
    # calculate entropy for the dataset (for testing/ verification)
    label_col = churnTrain_df.iloc[:,-1]
    _, n_counts = np.unique(label_col, return_counts=True)

    prob = n_counts / n_counts.sum()
    entropyOfDataset = sum(prob * -np.log2(prob))

    return entropyOfDataset

In [ ]:
def calculate_child_entropy(left, right):
    # calculate total number of rows
    n = len(left) + len(right)

    # calculate the occurences of each feature
    p_left =len(left) / n
    p_right = len(right) / n


    # calculate weighted entropy
    child_entropy = (p_left * calculate_child_entropy(left) + p_right * calculate_entropy(right))

In [18]:
def determine_best_split(data, possible_splits):
    
    # calculate the entropy of the data
    parent_entropy = calculate_entropy(data)
    best_entropy = parent_entropy

    # loop through all values for each column 
    for col_index in possible_splits:
        for value in possible_splits[col_index]:
            left, right = split_data(data, split_col=col_index, split_val=value)
            child_entropy = calculate_child_entropy(left, right)

            #c compare each entropy value and find lowest
            if child_entropy < best_entropy:
                best_split_col = col_index
                best_split_val = value
                best_entropy = child_entropy
                info_gain = parent_entropy - best_entropy

    return best_split_col, best_split_val

#### Function for calculating best split based on gini index<lowest>

    1. calculate gini index and gini gain for each feature
    2. compare gini gain in each feature
    3. feature with highest gini gain is selected as the splitting feature

In [ ]:
def calculate_gini(data):
    # calculate gini for target
    class_labels = np.unique(data)

    gini = 0

    for classes in class_labels:
        p_classes = len(data[data == classes]) / len(data)
        gini += p_classes ** 2

    return 1 - gini

In [ ]:
def calculate_gini_gain(parent, left, right):
    left_weight = len(left) / len(parent)
    right_weight = len(right) / len(parent)

    giniGain = calculate_gini(parent) - (left_weight* calculate_gini(left) + right_weight * calculate_gini(right))

    return giniGain

In [19]:
def determine_best_split2(data, possible_splits):
    label_col = data[:, -1]

    # initialise maz gini for comparision
    max_gini_gain = -float("inf")

    #dictionary to store possible splits and corresponding gini index
    gini = {}

    # loop through all values for each column 
    for col_index in possible_splits:
        for value in possible_splits[col_index]:
            left, right = split_data(data, split_col = col_index, split_val=value)
            if len(left) > 0 and len(right) > 0:
                y, left_y, right_y = data[:, -1], left[:, -1], right[:, -1]
                # calculate gini for each feature subset
                curr_gini_gain = calculate_gini_gain(y, left_y, right_y)

                # compare each subset's value
                if curr_gini_gain > max_gini_gain:
                    max_gini_gain = curr_gini_gain
                    best_split_col = col_index
                    best_split_val = value
    return best_split_col, best_split_val

#### Function for calculation best split based on gain ratio 

    1. calculate information gain and split info for each feature
    2. compare gain ratio in each feature
    3. feature with highest gain ratio is seltected as the splitting feature 

In [24]:
# Function to calculate split information for one feature
def calculate_Split_info(data_col):
    split_info = 0

    unique_vals, n_counts = np.unique(data_col, return_counts=True)
    prob = n_counts/ n_counts.sum()

    for idx, values in enumerate(unique_vals):

        occurences = n_counts[idx]
        prob = occurences/ n_counts.sum()
        temp = (prob* - np.log2(prob))
        split_info = split_info + temp
    
    return(split_info)

In [27]:
split_info = 0

unique_vals, n_counts = np.unique(churnTrain_df['CustomerID'], return_counts=True)
prob = n_counts/ n_counts.sum()

print(unique_vals)
print(n_counts)
print(n_counts.sum())

for idx, values in enumerate(unique_vals):

    occurences = n_counts[idx]
    prob = occurences/ n_counts.sum()
    temp = (prob* - np.log2(prob))
    split_info = split_info + temp

print(split_info)

[2.00000e+00 3.00000e+00 4.00000e+00 ... 4.49998e+05 4.49999e+05
         nan]
[1 1 1 ... 1 1 1]
440833
18.749872700174883


**Findings:**

>As all data are shown in 1, no duplicated data is found

#### Function for checking if we have reached a leaf node (tunning)

In [29]:
def check_leaf(data):
    ## store last column as label_col
    label_col = data[:, -1]

    # use np.unique() to extract out unique values from label_col
    classes = np.unique(label_col)

    if len(classes) == 1:
        return True
    else: 
        return False; 



In [65]:
data_copy = churnTrain_df.values
data_copy[:5]

array([[2.0, 30.0, 'Female', 39.0, 14.0, 5.0, 18.0, 'Standard', 'Annual',
        932.0, 17.0, 1.0],
       [3.0, 65.0, 'Female', 49.0, 1.0, 10.0, 8.0, 'Basic', 'Monthly',
        557.0, 6.0, 1.0],
       [4.0, 55.0, 'Female', 14.0, 4.0, 6.0, 18.0, 'Basic', 'Quarterly',
        185.0, 3.0, 1.0],
       [5.0, 58.0, 'Male', 38.0, 21.0, 7.0, 7.0, 'Standard', 'Monthly',
        396.0, 29.0, 1.0],
       [6.0, 23.0, 'Male', 32.0, 20.0, 5.0, 8.0, 'Basic', 'Monthly',
        617.0, 20.0, 1.0]], dtype=object)

In [66]:
np.unique(churnTest_df["Churn"], return_counts=True)

(array([0, 1], dtype=int64), array([33881, 30493], dtype=int64))

In [67]:
check_leaf(churnTrain_df.values)

False

# Function for Classification

Function gets the class for when a leaf node is reached

In [68]:
def classify_data(data):
    # since our labels are always in the last column 
    label_col = data[:, -1]
   
    # find each unique classes in the label column and how many times it appears
    classes, n_classes = np.unique(label_col, return_counts=True)

    # find index of the most frequent class
    index = n_classes.argmax()

    # extract out the name 
    classification = classes[index]

    return classification

In [69]:
classify_data(churnTrain_df.values)

0.0

Finding:

According to the above analysis, the most frequent data of last label column is '0.0'.

## Decision tree Classifier procedure

purning

In [87]:
def get_possible_splits(data):
    possible_splits = {}
    n_rows, n_cols = data.shape
    print("data shape:", data.shape)
    print("range(n_cols-1)", range(n_cols-1))
    for col_index in range(n_cols-1):
        possible_splits[col_index] = {}
        values = data[:, col_index]
        print("Values:", values)
        unique_values = np.unique(values)

        possible_splits[col_index] = unique_values

    return possible_splits

In [85]:
def decision_tree_classifier(df, max_depth, min_samples, counter = 0):

    # initialize variables at the start
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data= df.values
    else: 
        data = df

    print(len(data))

    # return data
    # early stopping conditions 
    if(check_leaf(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    else: 
        
        counter += 1
        possible_splits = get_possible_splits(data)

        # use determine_best_split which returns the feature/col with best info gain
        split_column, split_value = determine_best_split(data, possible_splits)

        #split the data according to the result in determine_best_split
        left, right = split_data(data, split_column, split_value)

        # choose question 
        feature_name = COLUMN_HEADERS[split_column]

        # format question 
        question = "{} = {}".format(feature_name, split_value)

        # initialise dictionary for storing tree
        # method will return this dictionary 
        sub_tree = {question: []}

        # recursive call with left and right branches
        yes_answer = decision_tree_classifier(left, max_depth, min_samples, counter)
        no_answer = decision_tree_classifier(right, max_depth, min_samples, counter)

        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)

        return sub_tree



##### Function for predicting values using the tree

In [76]:
def classify_one (test, tree):

    # save tree patern into list
    question = list(tree.keys())[0]

    # extract out feature label, operator'=' and values of the feature
    # using split
    feature_name, operator, value = question.split(" ")

    # asking the question 
    if str(test[feature_name]) == value: 
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    
    # if it is a dictionary, it means that the answer is a subtree.
    # then recursively call method until answer is found 
    # else, it is the answer
    # then return answer

    if not isinstance(answer, dict):
        return answer
    else:
        residual_tree = answer
        return classify_one(test, residual_tree)

In [77]:
def predict(test, tree):

    # initilaise array for storing 
    output = []
    index = test.index.tolist()
    for i in index:
        output.append(classify_one(test.loc[i], tree))

    return output

##### Function for calculating prediction accuracy

By comparing all predictions to its actual values, calculating the average mean of total predictions


In [78]:
def calculate_accuracy(df, tree): 
    df["classification"] = df.apply(classify_one, axis=1, args=(tree,))
    df["classification correct"] = df["classification"] == df["Class"]

    accuracy = df["classification_correct"].mean()

    return accuracy

#### Test out all the three models:

##### Test out first model - decision_tree_classifier()
        - split by info gain and entropy

In [ ]:
tree = decision_tree_classifier(churnTrain_df, max_depth=10, min_samples=10)
print(tree)
output = predict(churnTest_df, tree)